In [1]:
# !pip install pandas
# !pip install pinecone
# !pip unstall openai
# !pip install python-dotenv

In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

/Users/Alvin/Library/Caches/pypoetry/virtualenvs/shopping-assistant-w40lWNq4-py3.13/lib/python3.13/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
token= os.getenv("RUN_POD_TOKEN") 
open_ai_base_url = os.getenv("RUN_POD_EMDEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

## Try out embeddings

In [4]:
output = client.embeddings.create(input = ["helloo there"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.05535165220499039, -0.056572869420051575, 0.08585154265165329, -0.06237364932894707, 0.01749393157660961, -0.011418377049267292, 0.052420731633901596, 0.052603915333747864, 0.02895810268819332, -0.02263830602169037, -0.013250201940536499, -0.04787169769406319, 0.029446590691804886, 0.031751636415719986, 0.05635915696620941, -0.009006474167108536, 0.012746450491249561, -0.054222024977207184, -0.103620246052742, -0.019554734230041504, 0.029156550765037537, 0.05199330672621727, -0.028805451467633247, -0.034804679453372955, 0.0015770869795233011, -0.006640366278588772, 0.019661590456962585, 0.03562900051474571, -0.008487456478178501, -0.07376149296760559, -0.00048514746595174074, -0.012944898568093777, 0.04658942297101021, -0.0003446598129812628, 0.051382698118686676, 0.0005123385926708579, 0.059625908732414246, -0.025065474212169647, -0.07791363447904587, -0.005609964486211538, 0.0604502335190773, -0.025019679218530655, -0.0016944382805377245, -0.016669608652591705, 0.0225619804114103

In [5]:
len(embedings)

384

## Wrangle dataset

In [6]:
df=pd.read_json('products/products.jsonl',lines=True)

In [7]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [8]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [9]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [10]:
texts = df['text'].tolist()

In [11]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

In [12]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [13]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [14]:
output = client.embeddings.create(input = texts,model=model_name)

In [15]:
embeddings = output.data

In [16]:
embeddings

[Embedding(embedding=[-0.014077755622565746, -0.06458218395709991, 0.0038055721670389175, 0.004829929210245609, 0.030047805979847908, -0.04565923660993576, 0.037265993654727936, 0.010659417137503624, -0.027804521843791008, -0.050939347594976425, -0.05243487283587456, -0.033664532005786896, -0.0003986789961345494, -0.043766941875219345, 0.037510160356760025, -0.017106954008340836, -0.019563883543014526, -0.080636166036129, -0.07581386715173721, -0.007958319038152695, 0.04840611666440964, -0.061224885284900665, -0.10767766088247299, -0.03699130564928055, 0.035495784133672714, -0.048467155545949936, 0.08796117454767227, -0.019823310896754265, 0.001698677777312696, -0.13075144588947296, -0.01300189457833767, 0.009842983447015285, -0.02798764780163765, -0.025835925713181496, -0.035068489611148834, 0.0009804832516238093, 0.06757322698831558, -0.07593595236539841, 0.047093719244003296, 0.0005937264650128782, -0.0025122498627752066, 0.02977311797440052, -0.06409385055303574, -0.002041083760559

## Push data to database

In [17]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [18]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [19]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [20]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734870791,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 